In [ ]:
!pip install openai google_search_results serpapi python-dotenv apify aiohttp nest-asyncio

In [ ]:
import asyncio
import pandas as pd
import re
import time
import requests
import csv
import os
import aiohttp
import statistics
from datetime import datetime
from pydantic import BaseModel
from dotenv import load_dotenv
from openai import OpenAI
from serpapi import GoogleSearch
from typing import List, Dict, Any, Optional
from urllib.parse import urlparse
from apify_client import ApifyClient
import nest_asyncio

nest_asyncio.apply()

In [ ]:
load_dotenv()

# Set environment variables
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
CRUNCHBASE_API_KEY = os.getenv("CRUNCHBASE_API_KEY")
PERPLEXITY_API_KEY = os.getenv("PERPLEXITY_API_KEY")
APIFY_TOKEN = os.getenv ("APIFY_API_KEY")

# Initialize OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
class Profile(BaseModel):
    name: str
    url: str

class ProfileList(BaseModel):
    linkedin: List[Profile]

class SearchQueries(BaseModel):
    queries: List[str]

In [ ]:
def search_crunchbase_companies():
    """Search for AI companies in Crunchbase"""
    
    # API endpoint
    SEARCH_URL = f"https://api.crunchbase.com/v4/data/searches/organizations?user_key={CRUNCHBASE_API_KEY}"
    
    # UUIDs for filters
    location_uuid = "f110fca2-1055-99f6-996d-011c198b3928"  # United States
    category_uuids = [
        "c4d8caf3-5fe7-359b-f9f2-2d708378e4ee",  # artificial-intelligence
        "186d333a-99df-4a4a-6a0f-69bd2c0d0bba",  # intelligent-systems
        "5ea0cdb7-c9a6-47fc-50f8-c9b0fac04863"   # machine-learning
    ]
    
    # Request payload
    payload = {
        "field_ids": [
            "identifier", "categories", "location_identifiers", "short_description",
            "description", "funding_total", "founder_identifiers", "founded_on",
            "linkedin", "investor_identifiers", "last_equity_funding_type", "num_founders"
        ],
        "query": [
            {
                "type": "predicate",
                "field_id": "categories",
                "operator_id": "includes",
                "values": category_uuids
            },
            {
                "type": "predicate",
                "field_id": "location_identifiers",
                "operator_id": "includes",
                "values": [location_uuid]
            },
            {
                "type": "predicate",
                "field_id": "founded_on",
                "operator_id": "gte",
                "values": ["2010-01-01"]
            },
            {
                "type": "predicate",
                "field_id": "funding_total",
                "operator_id": "gte",
                "values": [{"currency": "USD", "value": 100000}]
            },
            {
                "type": "predicate",
                "field_id": "facet_ids",
                "operator_id": "includes",
                "values": ["company"]
            }
        ],
        "limit": 20,
    }
    
    # Make the POST request
    response = requests.post(SEARCH_URL, json=payload)
    
    if response.status_code == 200:
        results = response.json().get("entities", [])
        output_rows = []
        
        for item in results:
            props = item.get("properties", {})
            identifier = props.get("identifier", {})
            
            # Extract company data
            name = identifier.get("value", "")
            uuid = item.get("uuid", "")
            linkedin = props.get("linkedin", {}).get("value", "")
            short_description = props.get("short_description", "")
            full_description = props.get("description", "")
            created_at = props.get("founded_on", {}).get("value", "")
            
            # Founders
            founder_list = props.get("founder_identifiers", [])
            founders = "|".join([founder.get("value", "") for founder in founder_list])
            
            # Investors
            investor_list = props.get("investor_identifiers", [])
            investors = "|".join([inv.get("value", "") for inv in investor_list])
            
            # Categories
            categories = props.get("categories", [])
            category_list = "|".join([cat.get("value", "") for cat in categories])
            
            # Locations
            locations = props.get("location_identifiers", [])
            loc_dict = {loc["location_type"]: loc.get("value", "") for loc in locations}
            city = loc_dict.get("city", "")
            region = loc_dict.get("region", "")
            country = loc_dict.get("country", "")
            continent = loc_dict.get("continent", "")
            
            # Funding
            funding_total = props.get("funding_total", {}).get("value_usd", "")
            last_funding_type = props.get("last_equity_funding_type", "")
            
            # Append row
            output_rows.append({
                "uuid": uuid,
                "name": name,
                "linkedin": linkedin,
                "short_description": short_description,
                "full_description": full_description,
                "founded_on": created_at,
                "founders": founders,
                "investors": investors,
                "categories": category_list,
                "city": city,
                "region": region,
                "country": country,
                "continent": continent,
                "funding_total_usd": funding_total,
                "last_funding_type": last_funding_type
            })
        
        # Save to CSV
        df = pd.DataFrame(output_rows)
        df.to_csv("company_data.csv", index=False)
        print(f"✅ Results saved to 'company_data.csv' - {len(output_rows)} companies found")
        return df
    else:
        print(f"❌ Error {response.status_code}: {response.text}")
        return None

# Run the search
company_df = search_crunchbase_companies()

In [ ]:
def extract_name_from_url(url: str) -> str:
    """Extract name from LinkedIn URL"""
    parts = url.split('/')
    if 'in' in parts:
        name_part = parts[parts.index('in') + 1].replace('-', ' ').replace('%20', ' ')
        return name_part.title()
    return "Unknown"

def search_google(query: str, n: int = 10) -> List[Dict[str, str]]:
    """Search Google for LinkedIn profiles"""
    params = {
        "engine": "google",
        "q": query,
        "api_key": SERPER_API_KEY,
        "num": n
    }
    
    search = GoogleSearch(params)
    results = search.get_dict()
    organic_results = results.get("organic_results", [])
    
    return [
        {
            "title": r.get("title", ""),
            "link": r.get("link", ""),
            "snippet": r.get("snippet", "")
        }
        for r in organic_results
        if "linkedin.com/in/" in r.get("link", "")
    ]

def generate_search_queries(company: str, description: str, titles: List[str]) -> List[str]:
    """Generate optimized search queries for finding LinkedIn profiles"""
    prompt = f"""
You are a smart assistant designed to generate Google search queries to find LinkedIn profiles of specific roles at a company.

--- INPUT ---
Company Name: "{company}"
Company Description: "{description}"
Titles of interest: {', '.join(titles)}

--- OBJECTIVE ---
Create as few high-quality search queries as possible that:
1. Are optimized to find LinkedIn profiles of people holding the target roles at this exact company.
2. Use advanced search operators like site:linkedin.com/in
"""
    
    response = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        response_format=SearchQueries,
        temperature=0.0
    )
    return response.choices[0].message.parsed.queries

def extract_profiles_from_search(results: List[Dict[str, str]], company: str, description: str, titles: List[str]) -> List[Profile]:
    """Extract relevant profiles from search results"""
    combined = "\n".join(f"{r['title']} - {r['link']}\n{r['snippet']}" for r in results)
    prompt = f"""
You are a filtering assistant helping identify relevant LinkedIn profiles.

Company Name: {company}
Company Description: {description}
Roles of interest: {', '.join(titles)}

Below are search results from Google (including LinkedIn links). Your task is to extract only those LinkedIn profiles that are likely to belong to employees of this company in leadership roles like the ones mentioned.

Search Results:
{combined}
"""
    
    response = client.beta.chat.completions.parse(
        model="o3-mini",
        messages=[{"role": "user", "content": prompt}],
        response_format=ProfileList
    )
    return response.choices[0].message.parsed.linkedin

def extract_profiles_from_search_with_names(results: List[Dict[str, str]], company: str, description: str, titles: List[str], known_names: List[str]) -> List[Profile]:
    """Extract profiles using known founder names"""
    combined = "\n".join(f"{r['title']} - {r['link']}\n{r['snippet']}" for r in results)
    prompt = f"""
You are a filtering assistant helping identify LinkedIn profiles.

Company Name: {company}
Company Description: {description}
Known founder names: {', '.join(known_names)}
Roles of interest: {', '.join(titles)}

Below are search results from Google. Extract only those LinkedIn profiles that:
- Match the known names provided.
- Are likely to belong to this company.

Search Results:
{combined}
"""
    
    response = client.beta.chat.completions.parse(
        model="o3-mini",
        messages=[{"role": "user", "content": prompt}],
        response_format=ProfileList
    )
    return response.choices[0].message.parsed.linkedin

In [ ]:
def search_and_extract_profiles(queries: List[str], extractor_fn, n=10) -> List[Profile]:
    """Search and extract profiles using provided extractor function"""
    all_results = []
    for query in queries:
        all_results.extend(search_google(query, n))
    print(f"All results: {len(all_results)} profiles found")
    return extractor_fn(all_results)

def find_profiles(company: str, description: str, titles: List[str], known_names: List[str]) -> List[Profile]:
    """Main function to find LinkedIn profiles for a company"""
    
    if len(known_names) >= 4:
        print("Using known names approach (4+ founders)")
        queries = [f"{name} {company} site:linkedin.com/in" for name in known_names]
        print(f'Queries: {queries}')
        return search_and_extract_profiles(
            queries,
            lambda results: extract_profiles_from_search_with_names(results, company, description, titles, known_names),
            5
        )
    else:
        # Step 1: Try to use known names if any
        profiles = []
        if known_names:
            print(f"Using known names: {known_names}")
            queries = [f"{name} {company} site:linkedin.com/in" for name in known_names]
            profiles.extend(
                search_and_extract_profiles(
                    queries,
                    lambda results: extract_profiles_from_search_with_names(results, company, description, titles, known_names),
                    5
                )
            )
        
        # Step 2: Use generated queries to discover additional profiles
        print("Generating additional search queries...")
        generated_queries = generate_search_queries(company, description, titles)
        additional_profiles = search_and_extract_profiles(
            generated_queries,
            lambda results: extract_profiles_from_search(results, company, description, titles)
        )
        
        return profiles + additional_profiles

def process_crunchbase_csv(input_csv: str, output_csv: str):
    """Process Crunchbase data and find LinkedIn profiles"""
    df = pd.read_csv(input_csv)
    titles_to_search = ["CEO", "CTO", "Co-founder", "Vice President", "Founder"]
    output_rows = []
    
    for idx, row in df.iterrows():
        company = row['name']
        desc = row['full_description']
        existing_names = [n.strip() for n in str(row.get('founders', '')).split('|') if n.strip()]
        
        print(f"\n{'='*50}")
        print(f"Processing company: {company}")
        print(f"Existing founders: {existing_names}")
        
        try:
            profiles = find_profiles(company, desc, titles_to_search, existing_names)
            print(f"Found {len(profiles)} profiles")
            
            for profile in profiles:
                output_rows.append({
                    "person_name": profile.name,
                    "linkedin_url": profile.url,
                    **row.to_dict()
                })
        except Exception as e:
            print(f"Error processing {company}: {e}")
    
    # Save results
    df_new = pd.DataFrame(output_rows)
    df_filtered = df_new.drop_duplicates(subset=["name", "person_name"])
    df_filtered.to_csv(output_csv, index=False)
    print(f"\n✅ Profile search complete! Results saved to {output_csv}")
    return df_filtered

# Run profile search
profiles_df = process_crunchbase_csv("company_data.csv", "profiles_data.csv")

In [ ]:
class LinkedInScraper:
    """A class to scrape LinkedIn profiles using the Apify LinkedIn Profile Actor."""
    
    def __init__(
        self,
        apify_token: str,
        actor_id: str = "93aLP1rXaqE97cdr8",
        output_file: str = "linkedin_profiles.csv",
        wait_for_finish: bool = True,
        wait_interval: int = 15,
        timeout_seconds: int = 6000,
    ):
        self.client = ApifyClient(apify_token)
        self.actor_id = actor_id
        self.output_file = output_file
        self.wait_for_finish = wait_for_finish
        self.wait_interval = wait_interval
        self.timeout_seconds = timeout_seconds
    
    def validate_linkedin_url(self, url: str) -> bool:
        """Validate if the URL is a proper LinkedIn profile URL."""
        try:
            if not url:
                return False
            
            # Normalize URL - remove protocol and www for easier checking
            url_lower = url.lower()
            
            # Check if it contains linkedin.com and /in/
            if "linkedin.com" not in url_lower or "/in/" not in url_lower:
                return False
            
            # Parse the URL
            parsed_url = urlparse(url)
            
            # LinkedIn domains: linkedin.com, www.linkedin.com, *.linkedin.com (country codes)
            domain = parsed_url.netloc.lower()
            valid_domains = (
                domain == "linkedin.com" or 
                domain == "www.linkedin.com" or
                domain.endswith(".linkedin.com")
            )
            
            if not valid_domains:
                return False
                
            # Check path structure
            path_parts = parsed_url.path.strip("/").split("/")
            if len(path_parts) < 2 or path_parts[0] != "in":
                return False
                
            # Profile ID should exist and be reasonable length
            profile_id = path_parts[1]
            if not profile_id or len(profile_id) < 2:
                return False
                
            return True
        except:
            return False
    
    def prepare_input_data(self, urls: List[str]) -> List[str]:
        """Prepare the input data list for the Apify actor."""
        valid_urls = [url for url in urls if self.validate_linkedin_url(url)]
        if len(valid_urls) < len(urls):
            print(f"Warning: {len(urls) - len(valid_urls)} invalid URLs were removed.")
        return valid_urls
    
    def run_actor(self, profile_urls: List[str]) -> Dict[str, Any]:
        """Run the Apify actor with the given LinkedIn profile URLs."""
        if not profile_urls:
            raise ValueError("No valid LinkedIn URLs provided to run the actor.")
        
        run_input = {"profileUrls": profile_urls}
        print(f"Starting Apify actor ({self.actor_id}) for {len(profile_urls)} profiles...")
        
        run = self.client.actor(self.actor_id).call(run_input=run_input)
        return run
    
    def wait_for_run_to_finish(self, run_id: str) -> Optional[Dict[str, Any]]:
        """Poll the run status until it finishes or times out."""
        start = time.time()
        while True:
            run_info = self.client.run(run_id).get()
            status = run_info.get("status")
            if status in ("SUCCEEDED", "FAILED", "ABORTED"):
                return run_info
            elapsed = time.time() - start
            if elapsed > self.timeout_seconds:
                print(f"Timed out waiting for run {run_id} to finish (>{self.timeout_seconds}s).")
                return None
            print(f"Run {run_id} status: {status}. Checking again in {self.wait_interval}s...")
            time.sleep(self.wait_interval)
    
    def fetch_results(self, dataset_id: str) -> List[Dict[str, Any]]:
        """Fetch all items from the dataset produced by the actor run."""
        print(f"Fetching results from dataset {dataset_id}...")
        items = []
        for item in self.client.dataset(dataset_id).iterate_items():
            items.append(item)
        print(f"Fetched {len(items)} items.")
        return items
    
    def extract_earliest_year_from_caption(self, caption: str):
        """Extract earliest year from a caption string"""
        if not caption or not isinstance(caption, str):
            return None
        
        year_matches = [int(part) for part in caption.split() if part.isdigit() and 1900 <= int(part) <= datetime.now().year]
        if len(year_matches) >= 1:
            return min(year_matches)
        return None
    
    def extract_linkedin_profile_data(self, profiles: List[Dict]) -> List[Dict]:
        """Extracts structured data from a LinkedIn profile dictionary."""
        flat_data = []
        
        for profile in profiles:
            if not profile:
                continue
                
            base_entry = {
                'full_name': profile.get('fullName'),
                'headline': profile.get('headline'),
                'job_title': profile.get('jobTitle'),
                'company': profile.get('companyName'),
                'location': profile.get('addressWithCountry'),
                'linkedin_url': profile.get('linkedinUrl'),
                'about': profile.get('about'),
            }
            
            # Calculate estimated age based on experience and education
            experience_years = []
            for exp in profile.get('experiences', []):
                year = self.extract_earliest_year_from_caption(exp.get('caption'))
                if year:
                    experience_years.append(year)
            
            education_years = []
            for edu in profile.get('educations', []):
                year = self.extract_earliest_year_from_caption(edu.get('caption'))
                if year:
                    education_years.append(year)
            
            age = 30  # Default age
            if education_years or experience_years:
                earliest_edu = min(education_years) if education_years else float('inf')
                earliest_exp = min(experience_years) if experience_years else float('inf')
                
                if earliest_edu < earliest_exp:
                    age = datetime.now().year - earliest_edu + 18
                else:
                    age = datetime.now().year - earliest_exp + 22
            
            base_entry['estimated_age'] = age
            
            # Add up to 5 experiences
            for i in range(5):
                try:
                    base_entry[f'experience_{i+1}_title'] = profile['experiences'][i].get('title', '')
                    base_entry[f'experience_{i+1}_company'] = profile['experiences'][i].get('subtitle', '')
                except (IndexError, KeyError):
                    base_entry[f'experience_{i+1}_title'] = ''
                    base_entry[f'experience_{i+1}_company'] = ''
            
            # Add up to 3 educations
            for i in range(3):
                try:
                    base_entry[f'education_{i+1}_school'] = profile['educations'][i].get('title', '')
                    base_entry[f'education_{i+1}_degree'] = profile['educations'][i].get('subtitle', '')
                except (IndexError, KeyError):
                    base_entry[f'education_{i+1}_school'] = ''
                    base_entry[f'education_{i+1}_degree'] = ''
            
            # Add top 5 skills
            for i in range(5):
                try:
                    base_entry[f'skill_{i+1}'] = profile['skills'][i].get('title', '')
                except (IndexError, KeyError):
                    base_entry[f'skill_{i+1}'] = ''
            
            flat_data.append(base_entry)
        
        return flat_data
    
    def scrape_profiles(self, urls: List[str]) -> List[Dict[str, Any]]:
        """Scrape multiple LinkedIn profiles one by one"""
        valid_urls = self.prepare_input_data(urls)
        all_flattened_profiles = []
        
        for idx, url in enumerate(valid_urls, 1):
            print(f"Processing {idx}/{len(valid_urls)}: {url}")
            try:
                run = self.run_actor([url])  # Single URL wrapped in list
                run_id = run.get("id")
                dataset_id = run.get("defaultDatasetId")
                
                if self.wait_for_finish:
                    final_run_info = self.wait_for_run_to_finish(run_id)
                    if not final_run_info or final_run_info.get("status") != "SUCCEEDED":
                        print(f"Run {run_id} did not succeed or was aborted.")
                        continue
                
                if not dataset_id:
                    print(f"No dataset ID for run {run_id}.")
                    continue
                
                raw_results = self.fetch_results(dataset_id)
                flattened = self.extract_linkedin_profile_data(raw_results)
                all_flattened_profiles.extend(flattened)
                
            except Exception as e:
                print(f"❌ Error processing {url}: {e}")
        
        return all_flattened_profiles
    
    def enrich_and_update_existing_csv(self, input_csv_path: str):
        """Load a CSV with 'person_name' and 'linkedin_url' columns, scrape the profiles, and update the CSV."""
        df = pd.read_csv(input_csv_path)
        
        # Filter rows with valid LinkedIn URLs
        valid_mask = df["linkedin_url"].apply(self.validate_linkedin_url)
        if not valid_mask.any():
            print("No valid LinkedIn URLs found in the dataset.")
            return
        
        valid_df = df[valid_mask].copy()
        urls = valid_df["linkedin_url"].tolist()
        
        scraped_profiles = self.scrape_profiles(urls)
        if not scraped_profiles:
            print("No profile data returned.")
            return
        
        scraped_df = pd.DataFrame(scraped_profiles)
        # Merge on linkedin_url
        merged_df = pd.merge(df, scraped_df, how="left", left_on="linkedin_url", right_on="linkedin_url")
        
        merged_df.to_csv(input_csv_path, index=False)
        print(f"Updated dataset saved back to {input_csv_path}.")

In [ ]:
def run_linkedin_scraping():
    """Run the LinkedIn scraping process"""
    
    # Check if profiles_data.csv exists
    file_path = "profiles_data.csv"
    if not os.path.isfile(file_path):
        print(f"Error: {file_path} not found. Please run the profile search first.")
        return None
    
    # Initialize scraper
    scraper = LinkedInScraper(
        apify_token=APIFY_TOKEN,
        actor_id="2SyF0bVxmgGr8IVCZ"  # Updated actor ID
    )
    
    # Run scraping
    print("Starting LinkedIn profile scraping...")
    scraper.enrich_and_update_existing_csv(file_path)
    
    # Load and display results
    enriched_df = pd.read_csv(file_path)
    print(f"✅ Scraping complete! {len(enriched_df)} total records processed")
    return enriched_df

# Execute scraping (uncomment to run)
enriched_df = run_linkedin_scraping()

In [ ]:
class MetricsExtractor:
    def __init__(self, api_key, serper_key):
        self.api_key = api_key
        self.serper_key = serper_key
        self.session = None
        self.perplexity_url = "https://api.perplexity.ai/chat/completions"
        self.model = "sonar-reasoning"
    
    async def initialize(self):
        if not self.session:
            self.session = aiohttp.ClientSession()
    
    async def clean_cache(self):
        if self.session:
            await self.session.close()
        self.session = None
    
    async def search_info(self, query, year, region="us"):
        if not self.session:
            await self.initialize()
        
        # Add timestamp to avoid caching
        timestamp = int(time.time())
        query = f"{query} {timestamp}"
        
        payload = {
            "q": query,
            "gl": region,
            "num": 10,
            "page": 0
        }
        
        headers = {
            'X-API-KEY': self.serper_key,
            'Content-Type': 'application/json',
            'Cache-Control': 'no-cache'
        }
        
        try:
            async with self.session.post(
                "https://google.serper.dev/search",
                headers=headers,
                json=payload
            ) as response:
                if response.status == 200:
                    return await response.json()
                else:
                    print(f"Search API error: {response.status}")
                    return {"organic": []}
        except Exception as e:
            print(f"Search error: {str(e)}")
            return {"organic": []}
            
    async def get_market_metrics(self, sector_name, year):
        """Extract market size and CAGR metrics"""
        queries = [
            f"{sector_name} market size {year}",
            f"{sector_name} market value {year}",
            f"{sector_name} industry growth rate",
            f"{sector_name} market CAGR forecast",
            f"{sector_name} market analysis report"
        ]
        
        print("\nAnalyzing market metrics...")
        all_text = []
        seen_urls = set()
        
        for query in queries:
            try:
                results = await self.search_info(query, year)
                if isinstance(results, dict) and "organic" in results:
                    for item in results["organic"]:
                        url = item.get("link", "").strip()
                        if not url or url in seen_urls:
                            continue
                        seen_urls.add(url)
                        
                        text = f"{item.get('title', '')} {item.get('snippet', '')}"
                        if sector_name.lower() in text.lower():
                            all_text.append(text)
                            print(f"\nRelevant market data found:")
                            print(f"Title: {item.get('title', '')}")
                            print(f"Snippet: {item.get('snippet', '')}")
                            print(f"URL: {item.get('link', '')}")
            
            except Exception as e:
                print(f"Search error: {str(e)}")
                continue
        
        combined_text = " ".join(all_text)
        
        # Market size patterns
        market_size_patterns = [
            r'market\s+(?:size|value)\s+(?:of\s+)?(\$|\€|\£|USD|EUR|GBP|US$)\s*([\d,\.]+)\s*(billion|bn|b)',
            r'market\s+(?:size|value)\s+(?:of\s+)?(\$|\€|\£|USD|EUR|GBP|US$)\s*([\d,\.]+)\s*(million|mn|m)',
            r'(?:valued|estimated)\s+(?:at\s+)?(\$|\€|\£|USD|EUR|GBP|US$)\s*([\d,\.]+)\s*(billion|bn|b)',
            r'(?:valued|estimated)\s+(?:at\s+)?(\$|\€|\£|USD|EUR|GBP|US$)\s*([\d,\.]+)\s*(million|mn|m)',
            r'worth\s+(\$|\€|\£|USD|EUR|GBP|US$)\s*([\d,\.]+)\s*(billion|bn|b)',
            r'worth\s+(\$|\€|\£|USD|EUR|GBP|US$)\s*([\d,\.]+)\s*(million|mn|m)'
        ]
        
        # CAGR patterns
        cagr_patterns = [
            r'CAGR\s+of\s+([\d\.]+)\s*%',
            r'compound\s+annual\s+growth\s+rate\s+(?:\(CAGR\))?\s+of\s+([\d\.]+)\s*%',
            r'growth\s+rate\s+of\s+([\d\.]+)\s*%\s+(?:annually|per\s+year)'
        ]
        
        # Extract market sizes
        currency_rates = {
            '$': 1.0, 'USD': 1.0, 'US$': 1.0,
            '€': 1.1, 'EUR': 1.1,
            '£': 1.25, 'GBP': 1.25
        }
        market_sizes = []
        for pattern in market_size_patterns:
            matches = re.finditer(pattern, combined_text, re.IGNORECASE)
            for match in matches:
                try:
                    currency = match.group(1).upper()
                    value = float(match.group(2).replace(',', ''))
                    unit = match.group(3).lower()
                    if unit in ['million', 'mn', 'm']:
                        value /= 1000
                    fx = currency_rates.get(currency, 1)
                    value *= fx
                    
                    if 0 < value < 10000:
                        market_sizes.append(value)
                except:
                    continue
    
        # Extract CAGR values
        cagr_values = []
        for pattern in cagr_patterns:
            matches = re.finditer(pattern, combined_text, re.IGNORECASE)
            for match in matches:
                try:
                    value = float(match.group(1))
                    if 0 < value < 100:
                        cagr_values.append(value)
                except:
                    continue
        
        # Calculate metrics
        market_size = 0
        cagr = 0
        
        if market_sizes:
            market_size = statistics.median(market_sizes)
        
        if cagr_values:
            cagr = statistics.median(cagr_values)
        
        print("\nMarket Metrics Analysis:")
        print(f"Market sizes found: {market_sizes}")
        print(f"CAGR values found: {cagr_values}")
        print(f"\nFinal Metrics:")
        print(f"Market Size: ${market_size:.2f} billion")
        print(f"CAGR: {cagr:.1f}%")
        
        return {
            "market_size": market_size,
            "cagr": cagr
        }

In [ ]:
async def get_timing_score(self, sector_name, year, cagr):
    """Calculate market timing score based on multiple indicators"""
    # Search queries focused on timing indicators
    queries = [
        f"{sector_name} market readiness {year}",
        f"{sector_name} technology adoption stage {year}",
        f"{sector_name} market maturity analysis",
        f"{sector_name} industry growth phase",
        f"{sector_name} market barriers entry"
    ]
    
    print("\nAnalyzing market timing factors...")
    all_text = []
    seen_urls = set()
    
    # Collect timing data
    for query in queries:
        try:
            results = await self.search_info(query, year)
            if isinstance(results, dict) and "organic" in results:
                for item in results["organic"]:
                    url = item.get("link", "").strip()
                    if not url or url in seen_urls:
                        continue
                    seen_urls.add(url)
                    
                    text = f"{item.get('title', '')} {item.get('snippet', '')}"
                    if sector_name.lower() in text.lower():
                        all_text.append(text)
                        print(f"\nRelevant timing indicator found:")
                        print(f"Title: {item.get('title', '')}")
                        print(f"Snippet: {item.get('snippet', '')}")
                        print(f"URL: {item.get('link', '')}")
        except Exception as e:
            print(f"Search error: {str(e)}")
            continue
    
    combined_text = " ".join(all_text).lower()
    
    # Market phase indicators
    phase_indicators = {
        'early': {
            'terms': ['emerging', 'nascent', 'early stage', 'beginning', 'experimental',
                     'pilot', 'prototype', 'research phase', 'early adoption',
                     'innovative', 'breakthrough', 'pioneering', 'cutting edge'],
            'weight': 1.0
        },
        'growth': {
            'terms': ['growing', 'expansion', 'scaling', 'increasing adoption',
                     'rapid growth', 'accelerating', 'momentum', 'traction',
                     'market penetration', 'widespread adoption'],
            'weight': 0.8
        },
        'mature': {
            'terms': ['mature', 'established', 'stable', 'saturated', 'consolidated',
                     'mainstream', 'traditional', 'conventional', 'standardized'],
            'weight': 0.6
        }
    }
    
    # Calculate phase scores
    phase_scores = {phase: 0 for phase in phase_indicators}
    for phase, data in phase_indicators.items():
        score = 0
        for term in data['terms']:
            if term in combined_text:
                score += 1
        phase_scores[phase] = score * data['weight']
    
    # Normalize phase scores
    total_score = sum(phase_scores.values()) or 1
    phase_scores = {phase: score/total_score for phase, score in phase_scores.items()}
    
    # Market readiness indicators
    readiness_indicators = {
        'positive': {
            'terms': ['ready', 'prepared', 'opportunity', 'potential', 'promising',
                     'favorable', 'advantageous', 'strategic', 'beneficial'],
            'weight': 1.0
        },
        'negative': {
            'terms': ['barrier', 'challenge', 'obstacle', 'limitation', 'constraint',
                     'risk', 'concern', 'problem', 'issue', 'difficulty'],
            'weight': -0.5
        }
    }
    
    # Calculate readiness score
    readiness_score = 0
    for sentiment, data in readiness_indicators.items():
        for term in data['terms']:
            if term in combined_text:
                readiness_score += data['weight']
    
    # Normalize readiness score
    readiness_score = max(0, min(1, (readiness_score + 3) / 6))
    
    # Calculate timing score components
    emerging_tech = ['quantum', 'ai', 'blockchain', 'robotics']
    is_emerging = any(tech in sector_name.lower() for tech in emerging_tech)
    
    # Adjust CAGR expectations based on sector type
    if is_emerging:
        cagr_max = 60.0
    else:
        cagr_max = 40.0
    
    # Calculate normalized CAGR score
    cagr_score = min(cagr / cagr_max, 1.0)
    
    # Calculate final timing score
    phase_component = (phase_scores['early'] * 1.0 +
                     phase_scores['growth'] * 0.8 +
                     phase_scores['mature'] * 0.6)
    
    timing_score = (cagr_score * 0.4 +           # CAGR importance
                   phase_component * 0.3 +        # Market phase
                   readiness_score * 0.3) * 5     # Market readiness
    
    # Ensure score is within 1-5 range
    timing_score = round(max(1.0, min(5.0, timing_score)), 1)
    
    print("\nTiming Score Analysis:")
    print(f"CAGR Score: {cagr_score:.2f}")
    print(f"Market Phase Distribution: {phase_scores}")
    print(f"Market Readiness Score: {readiness_score:.2f}")
    print(f"\nFinal Timing Score: {timing_score}/5")
    
    return timing_score

# Add this method to MetricsExtractor class
MetricsExtractor.get_timing_score = get_timing_score

In [ ]:
async def get_regional_sentiment(self, sector_name, year, region):
    """Calculate regional market sentiment score"""
    # Region-specific configurations
    region_configs = {
        'us': {
            'country': 'United States',
            'region_terms': ['US', 'USA', 'United States', 'North America', 'American'],
            'gl': 'us',
            'cities': ['Silicon Valley', 'New York', 'Boston', 'Seattle', 'Austin']
        },
        'sg': {
            'country': 'Southeast Asia',
            'region_terms': ['Southeast Asia', 'ASEAN', 'Singapore', 'Indonesia', 'Malaysia', 'Thailand'],
            'gl': 'sg',
            'cities': ['Singapore', 'Jakarta', 'Bangkok', 'Kuala Lumpur', 'Ho Chi Minh']
        }
    }
    
    # Get region configuration
    region_config = region_configs.get(region.lower())
    if not region_config:
        print(f"Warning: Unsupported region '{region}'. Using default values.")
        return 3.0
    
    # Enhanced search queries
    queries = [
        f"{sector_name} market growth {region_config['country']} {year}",
        f"{sector_name} investment trends {region_config['country']}",
        f"{sector_name} adoption rate {region_config['country']}",
        f"{sector_name} market opportunity {region_config['country']}",
        f"{sector_name} industry development {region_config['country']}"
    ]
    
    print(f"\nAnalyzing {region_config['country']} market sentiment...")
    all_text = []
    seen_urls = set()
    
    # Collect regional data
    for query in queries:
        try:
            results = await self.search_info(query, year, region=region_config['gl'])
            if isinstance(results, dict) and "organic" in results:
                for item in results["organic"]:
                    url = item.get("link", "").strip()
                    if not url or url in seen_urls:
                        continue
                    seen_urls.add(url)
                    
                    text = f"{item.get('title', '')} {item.get('snippet', '')}"
                    if (sector_name.lower() in text.lower() and
                        any(term.lower() in text.lower() for term in region_config['region_terms'])):
                        all_text.append(text)
                        print(f"\nRelevant regional indicator found:")
                        print(f"Title: {item.get('title', '')}")
                        print(f"Snippet: {item.get('snippet', '')}")
                        print(f"URL: {item.get('link', '')}")
        except Exception as e:
            print(f"Search error: {str(e)}")
            continue
    
    combined_text = " ".join(all_text).lower()
    
    # Sentiment indicators
    sentiment_indicators = {
        'positive': {
            'terms': [
                'growth', 'opportunity', 'expansion', 'investment',
                'innovation', 'development', 'success', 'potential',
                'promising', 'favorable', 'strategic', 'advantage'
            ],
            'weight': 1.0
        },
        'negative': {
            'terms': [
                'challenge', 'barrier', 'limitation', 'restriction',
                'regulation', 'constraint', 'risk', 'concern',
                'problem', 'difficulty', 'obstacle', 'threat'
            ],
            'weight': -0.7
        }
    }
    
    # Calculate sentiment score
    sentiment_score = 0
    for sentiment, data in sentiment_indicators.items():
        for term in data['terms']:
            if term in combined_text:
                sentiment_score += data['weight']
    
    # Normalize sentiment score
    sentiment_score = max(0, min(1, (sentiment_score + 5) / 10))
    
    # Calculate regional presence
    presence_score = 0
    for city in region_config['cities']:
        if city.lower() in combined_text:
            presence_score += 0.2
    presence_score = min(1.0, presence_score)
    
    # Calculate final score
    final_score = (sentiment_score * 0.7 + presence_score * 0.3) * 5
    final_score = round(max(1.0, min(5.0, final_score)), 1)
    
    print(f"\nRegional Sentiment Analysis for {region_config['country']}:")
    print(f"Sentiment Score: {sentiment_score:.2f}")
    print(f"Regional Presence Score: {presence_score:.2f}")
    print(f"\nFinal Regional Sentiment Score: {final_score}/5")
    
    return final_score

# Add this method to MetricsExtractor class
MetricsExtractor.get_regional_sentiment = get_regional_sentiment

In [ ]:
async def get_competitor_metrics(self, sector_name, year):
    """Extract competitor metrics with enhanced validation"""
    # Comprehensive search queries
    queries = [
        f"{sector_name} market leaders revenue {year}",
        f"{sector_name} industry players market share {year}",
        f"{sector_name} companies competitive landscape {year}",
        f"{sector_name} market structure analysis {year}",
        f"{sector_name} venture funding deals {year}",
        f"{sector_name} startup investment rounds {year}",
        f"{sector_name} industry consolidation {year}"
    ]
    
    print("\nAnalyzing competitor metrics...")
    all_text = []
    seen_urls = set()
    
    # Collect competitor data
    for query in queries:
        try:
            results = await self.search_info(query, year)
            if isinstance(results, dict) and "organic" in results:
                for item in results["organic"]:
                    url = item.get("link", "").strip()
                    if not url or url in seen_urls:
                        continue
                    seen_urls.add(url)
                    text = f"{item.get('title', '')} {item.get('snippet', '')}"
                    if sector_name.lower() in text.lower():
                        all_text.append(text)
                        print(f"\nRelevant competitor data found:")
                        print(f"Title: {item.get('title', '')}")
                        print(f"Snippet: {item.get('snippet', '')}")
                        print(f"URL: {item.get('link', '')}")
        except Exception as e:
            print(f"Search error: {str(e)}")
            continue
    
    combined_text = " ".join(all_text)
    
    # Enhanced patterns
    patterns = {
        'competitors': [
            r'([\d,]+)\s+(?:major|key|leading|primary|main)\s+(?:players|companies|competitors)',
            r'(?:market|industry)\s+has\s+([\d,]+)\s+(?:players|companies|competitors)',
            r'top\s+([\d,]+)\s+(?:players|companies|competitors)',
            r'approximately\s+([\d,]+)\s+(?:players|companies|competitors)',
            r'more\s+than\s+([\d,]+)\s+(?:players|companies)',
            r'among\s+([\d,]+)\s+(?:players|companies|competitors)',
            r'comprises\s+([\d,]+)\s+(?:players|companies|competitors)'
        ],
        'funding': [
            r'[\$\€\£]\s*([\d,\.]+)\s*(billion|bn|b)\s+(?:funding|investment)',
            r'[\$\€\£]\s*([\d,\.]+)\s*(million|mn|m)\s+(?:funding|investment)',
            r'(?:funding|investment)\s+of\s+[\$\€\£]\s*([\d,\.]+)\s*(billion|bn|b)',
            r'(?:funding|investment)\s+of\s+[\$\€\£]\s*([\d,\.]+)\s*(million|mn|m)',
            r'raised\s+[\$\€\£]\s*([\d,\.]+)\s*(billion|bn|b)',
            r'raised\s+[\$\€\£]\s*([\d,\.]+)\s*(million|mn|m)'
        ]
    }
    
    # Extract competitor counts
    competitor_counts = []
    for pattern in patterns['competitors']:
        matches = re.finditer(pattern, combined_text, re.IGNORECASE)
        for match in matches:
            try:
                count = float(match.group(1).replace(',', ''))
                if 2 <= count <= 1000:  # Reasonable range
                    competitor_counts.append(count)
            except:
                continue
    
    # Extract funding values
    funding_values = []
    for pattern in patterns['funding']:
        matches = re.finditer(pattern, combined_text, re.IGNORECASE)
        for match in matches:
            try:
                value = float(match.group(1).replace(',', ''))
                unit = match.group(2).lower()
                if unit in ['million', 'mn', 'm']:
                    value /= 1000
                if 0 < value < 1000:  # Reasonable range in billions
                    funding_values.append(value)
            except:
                continue
    
    # Calculate metrics with validation
    def calculate_metric(values, default, min_valid=2):
        """Calculate metric with outlier removal"""
        if len(values) < min_valid:
            return default, 0.0
        
        if len(values) > 3:
            # Remove outliers
            q1, q3 = statistics.quantiles(values)[0], statistics.quantiles(values)[2]
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr
            values = [x for x in values if lower_bound <= x <= upper_bound]
        
        confidence = min(len(values) / 5, 1.0)
        return statistics.median(values) if values else default, confidence
    
    # Calculate final metrics
    competitor_count, comp_confidence = calculate_metric(competitor_counts, 10)
    competitor_count = int(competitor_count)
    total_funding, fund_confidence = calculate_metric(funding_values, 1.0)
    
    # Calculate momentum score
    try:
        emerging_tech = ['quantum computing', 'artificial intelligence', 'blockchain']
        is_emerging = any(tech in sector_name.lower() for tech in emerging_tech)
        
        if is_emerging:
            comp_scale = 35.0
            fund_scale = 25.0
            fund_weight = 0.7
        else:
            comp_scale = 25.0
            fund_scale = 15.0
            fund_weight = 0.6
        
        comp_weight = 1.0 - fund_weight
        
        norm_competitors = min(competitor_count / comp_scale, 1.0)
        norm_funding = min(total_funding / fund_scale, 1.0)
        
        total_confidence = (comp_confidence * comp_weight + fund_confidence * fund_weight)
        
        base_score = (norm_competitors * comp_weight + norm_funding * fund_weight) * 5
        momentum_score = base_score * total_confidence + 3.0 * (1 - total_confidence)
        momentum_score = round(max(1.0, min(5.0, momentum_score)), 1)
    except Exception as e:
        print(f"Error calculating momentum score: {str(e)}")
        momentum_score = 3.0
        total_confidence = 0.0
    
    print("\nCompetitor Metrics Analysis:")
    print(f"Competitor counts found: {competitor_counts}")
    print(f"Funding values found: {funding_values}")
    print(f"\nConfidence metrics:")
    print(f"Competitor confidence: {comp_confidence:.2f}")
    print(f"Funding confidence: {fund_confidence:.2f}")
    print(f"\nFinal metrics:")
    print(f"Competitor count: {competitor_count}")
    print(f"Total funding: ${total_funding:.2f}B")
    print(f"Momentum score: {momentum_score}/5")
    
    return {
        "competitor_count": competitor_count,
        "total_funding": total_funding,
        "momentum_score": momentum_score
    }

# Add this method to MetricsExtractor class
MetricsExtractor.get_competitor_metrics = get_competitor_metrics

In [ ]:
async def extract_all_metrics(self, sector_name, year):
    """Extract all metrics with enhanced error handling and parallel execution"""
    start_time = time.time()
    
    try:
        print(f"\nExtracting metrics for {sector_name} sector in {year}...")
        
        # Get market metrics first (market size and CAGR)
        print("\nFetching market metrics...")
        market_metrics = await self.get_market_metrics(sector_name, year)
        
        # Create tasks for parallel execution
        tasks = [
            self.get_timing_score(sector_name, year, market_metrics["cagr"]),
            self.get_regional_sentiment(sector_name, year, "us"),
            self.get_regional_sentiment(sector_name, year, "sg"),
            self.get_competitor_metrics(sector_name, year)
        ]
        
        # Execute remaining tasks in parallel
        timing_score, us_sentiment, sea_sentiment, competitor_metrics = await asyncio.gather(*tasks)
        
        # Combine all metrics with validation
        metrics = {
            "market_size": float(market_metrics.get("market_size", 0) or 0),
            "cagr": float(market_metrics.get("cagr", 0) or 0),
            "timing_score": float(timing_score or 3.0),
            "us_sentiment": float(us_sentiment or 3.0),
            "sea_sentiment": float(sea_sentiment or 3.0),
            "competitor_count": int(competitor_metrics.get("competitor_count", 10) or 10),
            "total_funding": float(competitor_metrics.get("total_funding", 1.0) or 1.0),
            "momentum_score": float(competitor_metrics.get("momentum_score", 3.0) or 3.0),
            "execution_time": time.time() - start_time
        }
        
        # Print summary
        print("\n===== METRICS EXTRACTION SUMMARY =====")
        print(f"Sector: {sector_name}")
        print(f"Year: {year}")
        print(f"\nMarket Metrics:")
        print(f"- Market Size: ${metrics['market_size']:.2f} billion")
        print(f"- CAGR: {metrics['cagr']:.1f}%")
        print(f"\nTiming and Sentiment:")
        print(f"- Market Timing Score: {metrics['timing_score']:.1f}/5")
        print(f"- US Market Sentiment: {metrics['us_sentiment']:.1f}/5")
        print(f"- SEA Market Sentiment: {metrics['sea_sentiment']:.1f}/5")
        print(f"\nCompetitor Metrics:")
        print(f"- Number of Competitors: {metrics['competitor_count']}")
        print(f"- Total Funding: ${metrics['total_funding']:.2f} billion")
        print(f"- Market Momentum Score: {metrics['momentum_score']:.1f}/5")
        print(f"\nExecution Time: {metrics['execution_time']:.2f} seconds")
        
        return metrics
    
    except Exception as e:
        print(f"\nError in extract_all_metrics: {str(e)}")
        # Return default values if something goes wrong
        return {
            "market_size": 0,
            "cagr": 0,
            "timing_score": 3.0,
            "us_sentiment": 3.0,
            "sea_sentiment": 3.0,
            "competitor_count": 10,
            "total_funding": 1.0,
            "momentum_score": 3.0,
            "execution_time": time.time() - start_time
        }

# Add this method to MetricsExtractor class
MetricsExtractor.extract_all_metrics = extract_all_metrics

In [ ]:
def extract_year(date_str):
    """Extract year from date string"""
    try:
        return pd.to_datetime(date_str).year
    except:
        return None

def clean_think(text):
    """Remove <think> tags from LLM output"""
    return re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL).strip()

async def get_sector_from_llm(description, session, api_key):
    """Get sector classification from LLM"""
    url = "https://api.perplexity.ai/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    messages = [{
        "role": "user",
        "content": f"""
Given the following company description, respond only with the industry sector name (no more than 10 words).
Do NOT include full sentences, company names, or explanations. Just return the sector name.

Description: {description}
"""
    }]
    payload = {
        "model": "sonar-reasoning",
        "messages": messages
    }
    
    try:
        async with session.post(url, headers=headers, json=payload) as resp:
            res = await resp.json()
            raw_output = res['choices'][0]['message']['content']
            return clean_think(raw_output)
    except Exception as e:
        print(f"Error getting sector from LLM: {e}")
        return "Technology"

def validate_data_integrity(df, required_columns):
    """Validate data integrity before processing"""
    missing_cols = [col for col in required_columns if col not in df.columns]
    if missing_cols:
        print(f"Warning: Missing columns: {missing_cols}")
        return False
    
    print(f"✅ Data validation passed. {len(df)} records with all required columns.")
    return True

def save_checkpoint(data, filename):
    """Save checkpoint data for recovery"""
    try:
        if isinstance(data, pd.DataFrame):
            data.to_csv(f"checkpoint_{filename}.csv", index=False)
        else:
            import pickle
            with open(f"checkpoint_{filename}.pkl", 'wb') as f:
                pickle.dump(data, f)
        print(f"✅ Checkpoint saved: checkpoint_{filename}")
    except Exception as e:
        print(f"❌ Error saving checkpoint: {e}")

def load_checkpoint(filename, file_type='csv'):
    """Load checkpoint data for recovery"""
    try:
        if file_type == 'csv':
            return pd.read_csv(f"checkpoint_{filename}.csv")
        else:
            import pickle
            with open(f"checkpoint_{filename}.pkl", 'rb') as f:
                return pickle.load(f)
    except FileNotFoundError:
        print(f"No checkpoint found: checkpoint_{filename}")
        return None
    except Exception as e:
        print(f"Error loading checkpoint: {e}")
        return None

In [ ]:
async def run_analysis_and_save(api_key, serper_key, input_file="profiles_data.csv", output_file="profiles_data_with_metrics.csv"):
    """Run the complete market analysis and save results"""
    
    # Check if input file exists
    if not os.path.isfile(input_file):
        print(f"Error: {input_file} not found. Please run the previous steps first.")
        return None
    
    # Load the data
    df = pd.read_csv(input_file)
    print(f"Loaded {len(df)} records from {input_file}")
    
    # Cache to store processed companies
    company_cache = {}
    
    async with aiohttp.ClientSession() as session:
        extractor = MetricsExtractor(api_key, serper_key)
        await extractor.initialize()
        
        # Get unique companies
        unique_companies = df['name'].unique() if 'name' in df.columns else df['name_x'].unique()
        print(f"Found {len(unique_companies)} unique companies to analyze")
        
        for idx, name in enumerate(unique_companies, 1):
            print(f"\n{'='*60}")
            print(f"PROCESSING COMPANY {idx}/{len(unique_companies)}: {name}")
            print(f"{'='*60}")
            
            # Get company data
            company_mask = (df['name'] == name) if 'name' in df.columns else (df['name_x'] == name)
            sub_df = df[company_mask]
            
            if len(sub_df) == 0:
                print(f"No data found for company: {name}")
                continue
            
            # Extract company information
            description = sub_df.iloc[0].get('full_description', '')
            founded = sub_df.iloc[0].get("founded_on", None)
            year = extract_year(founded) or 2020  # Default to 2020 if no date
            
            if not description:
                print(f"No description found for {name}, skipping...")
                continue
            
            try:
                # Get sector classification
                print("Getting sector classification...")
                sector = await get_sector_from_llm(description, session, api_key)
                print(f"Classified sector: {sector}")
            except Exception as e:
                print(f"Error getting sector classification: {e}")
                sector = "Technology"
            
            print(f"Analyzing: {name}")
            print(f"Sector: {sector} | Founded: {year}")
            print(f"Description: {description[:200]}...")
            
            try:
                # Extract all metrics
                metrics = await extractor.extract_all_metrics(sector, year)
                print(f"✅ Successfully extracted metrics for {name}")
            except Exception as e:
                print(f"❌ Error extracting metrics for {name}: {e}")
                # Use default values
                metrics = {
                    "market_size": 0, "cagr": 0, "timing_score": 3,
                    "us_sentiment": 3, "sea_sentiment": 3,
                    "competitor_count": 10, "total_funding": 1.0,
                    "momentum_score": 3.0, "execution_time": 0
                }
            
            # Store in cache
            company_cache[name] = {
                "Sector": sector,
                "Year": year,
                **metrics
            }
            
            print(f"✅ Completed analysis for {name}")
        
        await extractor.clean_cache()
    
    # Add the metrics to the original DataFrame
    name_col = 'name' if 'name' in df.columns else 'name_x'
    
    for col in ["Sector", "Year", "market_size", "cagr", "timing_score", "us_sentiment",
                "sea_sentiment", "competitor_count", "total_funding", "momentum_score", "execution_time"]:
        df[col] = df[name_col].map(lambda name: company_cache.get(name, {}).get(col))
    
    # Save results
    df.to_csv(output_file, index=False)
    print(f"\n🎉 Analysis complete! Results saved to {output_file}")
    print(f"Total companies analyzed: {len(company_cache)}")
    print(f"Total records with metrics: {len(df)}")
    
    return df

# This function will be called in the next cell

In [ ]:
async def main():
    """Main execution function"""
    print("🚀 Starting Complete Market Analysis Pipeline")
    print("=" * 60)
    
    try:
        # Run the complete analysis
        result_df = await run_analysis_and_save(
            api_key=PERPLEXITY_API_KEY,
            serper_key=SERPER_API_KEY,  # Updated serper key
            input_file="profiles_data.csv",
            output_file="profiles_data_with_metrics.csv"
        )
        
        if result_df is not None:
            print("\n📊 Analysis Summary:")
            print(f"Total records processed: {len(result_df)}")
            print(f"Companies with metrics: {result_df['Sector'].notna().sum()}")
            
            # Show sample of results
            print("\n📋 Sample Results:")
            sample_cols = ['name', 'Sector', 'market_size', 'cagr', 'timing_score', 'us_sentiment']
            if 'name_x' in result_df.columns:
                sample_cols[0] = 'name_x'
            
            available_cols = [col for col in sample_cols if col in result_df.columns]
            print(result_df[available_cols].head(3).to_string(index=False))
            
            return result_df
        else:
            print("❌ Analysis failed")
            return None
            
    except Exception as e:
        print(f"❌ Error in main execution: {str(e)}")
        return None

# Execute the analysis
final_results = await main()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def analyze_results(csv_file="profiles_data_with_metrics.csv"):
    """Analyze and visualize the results"""
    
    if not os.path.isfile(csv_file):
        print(f"Results file {csv_file} not found. Please run the analysis first.")
        return
    
    # Load results
    df = pd.read_csv(csv_file)
    print(f"Loaded {len(df)} records from {csv_file}")
    
    # Filter out rows without metrics
    df_with_metrics = df.dropna(subset=['Sector', 'market_size'])
    print(f"Records with complete metrics: {len(df_with_metrics)}")
    
    if len(df_with_metrics) == 0:
        print("No records with metrics found.")
        return
    
    # Set up the plotting style
    plt.style.use('default')
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Market Analysis Dashboard', fontsize=16, fontweight='bold')
    
    # 1. Market Size by Sector
    sector_market = df_with_metrics.groupby('Sector')['market_size'].mean().sort_values(ascending=False)
    axes[0, 0].bar(range(len(sector_market)), sector_market.values)
    axes[0, 0].set_title('Average Market Size by Sector')
    axes[0, 0].set_ylabel('Market Size (Billions USD)')
    axes[0, 0].set_xticks(range(len(sector_market)))
    axes[0, 0].set_xticklabels(sector_market.index, rotation=45, ha='right')
    
    # 2. CAGR Distribution
    axes[0, 1].hist(df_with_metrics['cagr'], bins=10, alpha=0.7, color='skyblue')
    axes[0, 1].set_title('CAGR Distribution')
    axes[0, 1].set_xlabel('CAGR (%)')
    axes[0, 1].set_ylabel('Frequency')
    
    # 3. Timing Score vs Market Size
    axes[0, 2].scatter(df_with_metrics['timing_score'], df_with_metrics['market_size'], alpha=0.6)
    axes[0, 2].set_title('Timing Score vs Market Size')
    axes[0, 2].set_xlabel('Timing Score')
    axes[0, 2].set_ylabel('Market Size (Billions USD)')
    
    # 4. Regional Sentiment Comparison
    sentiment_data = df_with_metrics[['us_sentiment', 'sea_sentiment']].mean()
    axes[1, 0].bar(['US Market', 'SEA Market'], sentiment_data.values, color=['blue', 'orange'])
    axes[1, 0].set_title('Average Regional Sentiment')
    axes[1, 0].set_ylabel('Sentiment Score')
    axes[1, 0].set_ylim(0, 5)
    
    # 5. Competitor Count vs Funding
    axes[1, 1].scatter(df_with_metrics['competitor_count'], df_with_metrics['total_funding'], alpha=0.6)
    axes[1, 1].set_title('Competitor Count vs Total Funding')
    axes[1, 1].set_xlabel('Number of Competitors')
    axes[1, 1].set_ylabel('Total Funding (Billions USD)')
    
    # 6. Momentum Score Distribution
    axes[1, 2].hist(df_with_metrics['momentum_score'], bins=10, alpha=0.7, color='lightgreen')
    axes[1, 2].set_title('Market Momentum Score Distribution')
    axes[1, 2].set_xlabel('Momentum Score')
    axes[1, 2].set_ylabel('Frequency')
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print("\n📊 SUMMARY STATISTICS")
    print("=" * 50)
    
    numeric_cols = ['market_size', 'cagr', 'timing_score', 'us_sentiment', 
                   'sea_sentiment', 'competitor_count', 'total_funding', 'momentum_score']
    
    for col in numeric_cols:
        if col in df_with_metrics.columns:
            print(f"\n{col.upper().replace('_', ' ')}:")
            print(f"  Mean: {df_with_metrics[col].mean():.2f}")
            print(f"  Median: {df_with_metrics[col].median():.2f}")
            print(f"  Std: {df_with_metrics[col].std():.2f}")
            print(f"  Min: {df_with_metrics[col].min():.2f}")
            print(f"  Max: {df_with_metrics[col].max():.2f}")
    
    # Top performers
    print("\n🏆 TOP PERFORMERS")
    print("=" * 50)
    
    if len(df_with_metrics) > 0:
        name_col = 'name' if 'name' in df_with_metrics.columns else 'name_x'
        
        print("\nTop 5 by Market Size:")
        top_market = df_with_metrics.nlargest(5, 'market_size')[name_col].tolist()
        for i, company in enumerate(top_market, 1):
            print(f"  {i}. {company}")
        
        print("\nTop 5 by CAGR:")
        top_cagr = df_with_metrics.nlargest(5, 'cagr')[name_col].tolist()
        for i, company in enumerate(top_cagr, 1):
            print(f"  {i}. {company}")
        
        print("\nTop 5 by Timing Score:")
        top_timing = df_with_metrics.nlargest(5, 'timing_score')[name_col].tolist()
        for i, company in enumerate(top_timing, 1):
            print(f"  {i}. {company}")

# Run analysis (uncomment to execute)
analyze_results()